# Symbolic calculation of q3i and q1i

This system is composed of three disks, linked to three arms and a platform in the end. Le goal is to orientate the platform, so the disks do a rotation following a circle called "proximal circle". Then, these disks make the arm rotate around the platform's center on a circle called "distal circle".

<img src="screenshot.png" alt="spherical joint" width="600"/>

The angles reached by the arms are called "q3i" and those made by disks are called "q1i".

## Importation of useful librairies


In [4]:
#Maths operations librairies
import numpy as np
import quaternion
import math
from numpy import linalg as LA

#Symbolic operations librairies
from sympy.solvers import solve
from sympy import symbols,cos,sin,sqrt,acos,simplify
from sympy.algebras.quaternion import Quaternion

## Initial frame R0

The initial frame R0 is expressed by the vectors x0, y0 and z0 
that are converted to quaternion type

In [2]:
x0 = [1, 0, 0]
y0 = [0, 1, 0]
z0 = [0, 0, 1]

x0_quat = Quaternion(0,1,0,0)
y0_quat = Quaternion(0,0,1,0)
z0_quat = Quaternion(0,0,0,1)

## Parameters
The angles of the disks depend on several parameters as : <br>
- R : radius of the distal circle
- Rp : radius of the proximal circle
- Pc : pcx, pcy, pcz coordinates of the center of the distal link in 3D space
- Cp : cx, cy, cz coordinates of the center of the proximal link <br>

The angles of the disks on the proximal circle are expressed as q1 and those on the distal links are expressed as q3.<br>

In [ ]:
pcx,pcy,pcz = symbols('pcx pcy pcz')
cx,cy,cz = symbols('cx cy cz')
R,Rp = symbols('R Rp')#Radius of the distal and proximal circle



Pc = [0,0,pcz]   #Coordinates of the plateform's center
C = [0,0,cz]   #Coordinates of the center of the proximal circle

q3,q1 = symbols('q3 q1')

The vectors of the platform's frame are X, Y and Z and are defined as follows : <br>

In [ ]:
X1,X2,X3,Y1,Y2,Y3,Z1,Z2,Z3 = symbols('X1 X2 X3 Y1 Y2 Y3 Z1 Z2 Z3')

X_ = [X1,X2,X3]
Y_ = [Y1,Y2,Y3]
Z_ = [Z1,Z2,Z3]

## Math equations

The equation of each distal circle can be expressed with rotation the matrix Rt which describes the rotation with Euler's angles, and the Rz(theta) matrix which corresponds to a rotation of angle theta around the Z axis of the platform:

$Xi=Pc+Rcos(q3i)RtRz(theta)Z0+Rsin(q3i)RtRz(theta)X0$ (Eq. 1)

The proximal circle also has its own equation:

$Xip=Cp+Rpcos(q1i)X0+Rpsin(q1i)Y0$  (Eq. 2)

In the equation (Eq. 1), Rt*Rz(theta)*Z0 corresponds to the final Z after rotation, and Rt*Rz(theta)*X0 to the final X axis. These final Z and X axis can be calculated with quaternions instead of rotation matrix.

To find the expressions of q3i and q1i, the equation (Eq. 1)=(Eq. 2) must be solved.

$Pc+R*cos(q3i)*Zfinal+R*sin(q3i)*Xfinal=Cp+Rp*cos(q1i)*X0+Rp*sin(q1i)*Y0$

Since this equation has 3D vectors, a system with 3 equations and 3 unknowns (q3i and q1i) must be solved:

$R*cos(q3i)*Z1+R*sin(q3i)*X1=Rp*cos(q1i)$<br>
$R*cos(q3i)*Z2+R*sin(q3i)*X2=Rp*sin(q1i)$<br>
$Pcz+R*cos(q3i)*Z3+R*sin(q3i)*Z3=Cpz$<br>


## Solver

Only the third equation must be solved to find q3i.<br>
To do so, the scipy librairie is used to solve symbolic equations.<br>

First, the system of 3 equations is built.

In [ ]:
eq = []

for i in range(3):
    eq.append(Pc[i]+R*cos(q3)*Z_[i]+R*sin(q3)*X_[i]-(C[i]+Rp*cos(q1)*x0[i]+Rp*sin(q1)*y0[i])) 
    simplify(eq[i])
print("eq : ",eq)  #Equation to solve

Then, the solutions of the equation are calculated with the scipy function "solve" and put into a variable that is converted in a list which can be printed:

In [6]:
A = solve(eq[2],q3)   #Solving the 3rd equation gives the q31 solution
A_list = []
for i in A:
    A_list.append(i)
print("q3 : ",A_list)

q3 :  [2*atan((R*X3 - sqrt(R**2*X3**2 + R**2*Z3**2 - cz**2 + 2*cz*pcz - pcz**2))/(R*Z3 + cz - pcz)), 2*atan((R*X3 + sqrt(R**2*X3**2 + R**2*Z3**2 - cz**2 + 2*cz*pcz - pcz**2))/(R*Z3 + cz - pcz))]


## Finding q1i

This kind of algorithm can be used to find q1i, but the expression found is much more complicated than it really is. This is why q1i has been calculated by hand :

$R*cos(q3i)*Z1+R*sin(q3i)*X1=Rp*cos(q1i)$<br>
$R*cos(q3i)*Z2+R*sin(q3i)*X2=Rp*sin(q1i)$<br>

So, 

$R*[cos(q3i)*Z1+sin(q3i)*X1]=Rp*cos(q1i)$<br>
$R*[cos(q3i)*Z2+sin(q3i)*X2]=Rp*sin(q1i)$<br>

So,

$tan(q1i) = \frac{(cos(q3i)*Z2 + sin(q3i)*X2)}{(cos(q3i)*Z1 + sin(q3i)*X1)}$

Finally:

$q1i = atan2(cos(q3i)*Z2 + sin(q3i)*X2,cos(q3i)*Z1 + sin(q3i)*X1)$

# Useless codes
## Find symbolic expression of the new frame X, Y and Z

In [ ]:
### EXPRESSION OF NEW FRAME ###

x0 = [1, 0, 0]
y0 = [0, 1, 0]
z0 = [0, 0, 1]

x0_quat = Quaternion(0,1,0,0)
y0_quat = Quaternion(0,0,1,0)
z0_quat = Quaternion(0,0,0,1)

beta,a,b,c = symbols('beta a b c')


### GOAL VECTOR (the desired Z axis)###
goal = [a,b,c]


goal_norm = []
for i in goal:
    goal_norm.append(i/sqrt(goal[0]**2+goal[1]**2+goal[2]**2)) #Normalized vector of goal
    
### VECTOR AND ANGLE OF ROTATION ###
vector= np.cross(z0,goal_norm)


vector_norm = [] #Normalized vector of rotation
for i in vector:
    vector_norm.append(i/sqrt(vector[0]**2+vector[1]**2+vector[2]**2))
print("vector = ",vector_norm)
alpha = acos(np.vdot(z0,goal_norm)) #Angle of rotation
print("alpha = ",alpha)

if alpha == 0 :
    v = [0.0,0.0,1.0]
else:
    v = [vector_norm[0],vector_norm[1],vector_norm[2]]
    
### QUATERNION OF ROTATION ###
w1 = cos(alpha/2.0)
x1 = sin(alpha/2.0)*v[0]
y1 = sin(alpha/2.0)*v[1]
z1 = sin(alpha/2.0)*v[2]

q = Quaternion(w1,x1,y1,z1) #1st rotation quaternion
q_inv = Quaternion(w1,-x1,-y1,-z1)

z_prime = q.mul(z0_quat.mul(q_inv))

w2 = cos(beta/2.0)
x2 = sin(beta/2.0)*z_prime.b
y2 = sin(beta/2.0)*z_prime.c
z2 = sin(beta/2.0)*z_prime.d

q2 = Quaternion(w2,x2,y2,z2) #Quaternion of the rotation on new z axis
q2_inv = Quaternion(w2,-x2,-y2,-z2)

new_z = q2.mul(z_prime.mul(q2_inv)) #Final Z
new_x = q2.mul((q.mul(x0_quat.mul(q_inv))).mul(q2_inv)) #Final X
new_y = q2.mul((q.mul(y0_quat.mul(q_inv))).mul(q2_inv)) #Final Y

X = [simplify(new_x.b), simplify(new_x.c), simplify(new_x.d)]
Y = [new_y.b, new_y.c, new_y.d]
Z = [new_z.b, new_z.c, new_z.d]

print("X = ",X)

## The two others symbolic equations solving to find q1i 

In [10]:
for i in range(2):
    eq[i]=(Pc[i]+R*cos(A_list[0])*Z_[i]+R*sin(A_list[0])*X_[i]-(C[i]+Rp*cos(q1)*x0[i]+Rp*sin(q1)*y0[i])) 
    simplify(eq[i])

res = solve([eq[0],eq[1]],q1)
res_list1=[]
for i in range(len(res)):
    res_list1.append([])
    for j in res[i]:
        res_list1[i].append(j)   #Converts the results in a list
print("q1 : ",res_list1)

for i in range(2):
    eq[i]=(Pc[i]+R*cos(A_list[1])*Z_[i]+R*sin(A_list[1])*X_[i]-(C[i]+Rp*cos(q1)*x0[i]+Rp*sin(q1)*y0[i])) 
    simplify(eq[i])

res = solve([eq[0],eq[1]],q1)
res_list2=[]
for i in range(len(res)):
    res_list2.append([])
    for j in res[i]:
        res_list2[i].append(j)   #Converts the results in a list
print("q1 : ",res_list2)



q1 :  [[-acos(R*(X1*sin(2*atan(R*X3/(R*Z3 + cz - pcz) - sqrt(R**2*X3**2 + R**2*Z3**2 - cz**2 + 2*cz*pcz - pcz**2)/(R*Z3 + cz - pcz))) + Z1*cos(2*atan(R*X3/(R*Z3 + cz - pcz) - sqrt(R**2*X3**2 + R**2*Z3**2 - cz**2 + 2*cz*pcz - pcz**2)/(R*Z3 + cz - pcz))))/Rp) + 2*pi], [acos(R*(X1*sin(2*atan(R*X3/(R*Z3 + cz - pcz) - sqrt(R**2*X3**2 + R**2*Z3**2 - cz**2 + 2*cz*pcz - pcz**2)/(R*Z3 + cz - pcz))) + Z1*cos(2*atan(R*X3/(R*Z3 + cz - pcz) - sqrt(R**2*X3**2 + R**2*Z3**2 - cz**2 + 2*cz*pcz - pcz**2)/(R*Z3 + cz - pcz))))/Rp)]]
q1 :  [[-acos(R*(X1*sin(2*atan(R*X3/(R*Z3 + cz - pcz) + sqrt(R**2*X3**2 + R**2*Z3**2 - cz**2 + 2*cz*pcz - pcz**2)/(R*Z3 + cz - pcz))) + Z1*cos(2*atan(R*X3/(R*Z3 + cz - pcz) + sqrt(R**2*X3**2 + R**2*Z3**2 - cz**2 + 2*cz*pcz - pcz**2)/(R*Z3 + cz - pcz))))/Rp) + 2*pi], [acos(R*(X1*sin(2*atan(R*X3/(R*Z3 + cz - pcz) + sqrt(R**2*X3**2 + R**2*Z3**2 - cz**2 + 2*cz*pcz - pcz**2)/(R*Z3 + cz - pcz))) + Z1*cos(2*atan(R*X3/(R*Z3 + cz - pcz) + sqrt(R**2*X3**2 + R**2*Z3**2 - cz**2 + 2*cz*pcz